<b>Quiz 3 - Problem 2</b>

In [1]:
# data

import csv
f = open("quiz 3 problem 2.csv")
csvfile = csv.DictReader(f, delimiter=',')
headers = csvfile.fieldnames

table = []
for row in csvfile:
    table.append(row)
    
f.close()

# Create set of Nodes.
Nodes = set()
# Create dictionary "Distance" for each arc.
# We can borrow the keys from this dictionary to define the arc set of the network.
Distance = {}
for row in table:
    fromNode = row['From']
    toNode = row['To']
    Distance[(fromNode,toNode)] = float(row['Distance (miles)'])
    Nodes.add(fromNode)
    Nodes.add(toNode)

Arcs = Distance.keys()

In [2]:
Commodities = {'C1', 'C2'}
Origin = { 'C1' : '1', 'C2' : '2'}
Destination = { 'C1' : 'P', 'C2' : 'P'}
Volume = { 'C1' : 1, 'C2' : 1}

In [3]:

Capacity = {}

# Loop through the arcs and insert values into the dictionary
for arc in Arcs:
    if arc in [('5', '8'), ('6', '9'), ('7', '10')]:
        Capacity[arc] = 1
    else:
        Capacity[arc] = 2

# Print the resulting dictionary
print(Capacity)


{('1', '3'): 2, ('2', '4'): 2, ('3', '4'): 2, ('3', '5'): 2, ('3', '6'): 2, ('4', '3'): 2, ('4', '6'): 2, ('4', '7'): 2, ('5', '8'): 1, ('6', '9'): 1, ('7', '10'): 1, ('8', '11'): 2, ('8', '12'): 2, ('9', '11'): 2, ('9', '12'): 2, ('10', '12'): 2, ('11', 'P'): 2, ('12', 'P'): 2}


In [4]:
# define supply/demand for each node and each commodity
Supply = {}
for i in Nodes:
    for k in Commodities:
        if i==Origin[k]:
            Supply[i,k] = -Volume[k]
        elif i==Destination[k]:
            Supply[i,k] = Volume[k]
        else:
            Supply[i,k] = 0

In [5]:
# define a set of arc/commodity pairs (useful for variables)
ArcCommodities = { (i,j,k) for (i,j) in Arcs for k in Commodities }

In [6]:
from docplex.mp.model import Model
mdl = Model()

In [7]:
shipped = mdl.continuous_var_dict(ArcCommodities, lb=0, name='shipped')

In [8]:
# objective
mdl.minimize(mdl.sum(Distance[i,j]*shipped[i,j,k] for (i,j,k) in shipped))

In [9]:
# flow balance constraints for each commodity and for each node
for k in Commodities:
    for j in Nodes:
        inflow = sum(shipped[i,j,k] for i in Nodes if (i,j) in Arcs)
        outflow = sum(shipped[j,i,k] for i in Nodes if (j,i) in Arcs)
        mdl.add_constraint(inflow - outflow == Supply[j,k])

In [10]:
# define arc capacity constraints (over all commodities)
for (i,j) in Arcs:
    mdl.add_constraint(mdl.sum(shipped[i,j,k] for k in Commodities) <= Capacity[i,j])

In [11]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0,status='optimal')

In [12]:
mdl.print_solution()

objective: 585.000
status: OPTIMAL_SOLUTION(2)
  shipped_8_11_C1=1.000
  shipped_2_4_C2=1.000
  shipped_11_P_C1=1.000
  shipped_4_6_C2=1.000
  shipped_5_8_C1=1.000
  shipped_3_5_C1=1.000
  shipped_12_P_C2=1.000
  shipped_9_12_C2=1.000
  shipped_1_3_C1=1.000
  shipped_6_9_C2=1.000


In [13]:
# Extracting the solution and determining the paths
path_1 = ['1']
path_2 = ['2']

current_node_1 = '1'
current_node_2 = '2'

# Trace the path for shipment C1
while current_node_1 != 'P':
    for (i, j, k) in shipped:
        if k == 'C1' and i == current_node_1 and shipped[i, j, k].solution_value > 0:
            path_1.append(j)
            current_node_1 = j
            break

# Trace the path for shipment C2
while current_node_2 != 'P':
    for (i, j, k) in shipped:
        if k == 'C2' and i == current_node_2 and shipped[i, j, k].solution_value > 0:
            path_2.append(j)
            current_node_2 = j
            break

print("Route 1:", ' -> '.join(path_1))
print("Route 2:", ' -> '.join(path_2))

Route 1: 1 -> 3 -> 5 -> 8 -> 11 -> P
Route 2: 2 -> 4 -> 6 -> 9 -> 12 -> P


In [14]:
import csv


# Read the CSV file
f = open("quiz 3 problem 2.csv")
csvfile = csv.DictReader(f, delimiter=',')
headers = csvfile.fieldnames

# Parse the CSV file
table = []
for row in csvfile:
    table.append(row)
f.close()

# Create set of Nodes
Nodes = set()
# Create dictionary "Distance" for each arc
Distance = {}
for row in table:
    fromNode = row['From']
    toNode = row['To']
    if (fromNode, toNode) != ('5', '8'):  # Exclude arc (5, 8)
        Distance[(fromNode, toNode)] = float(row['Distance (miles)'])
        Nodes.add(fromNode)
        Nodes.add(toNode)

# Define arcs and commodities
Arcs = Distance.keys()
Commodities = {'C1', 'C2'}
Origin = {'C1': '1', 'C2': '2'}
Destination = {'C1': 'P', 'C2': 'P'}
Volume = {'C1': 1, 'C2': 1}

# Define capacity for arcs
Capacity = {}
for arc in Arcs:
    if arc in [('6', '9'), ('7', '10')]:
        Capacity[arc] = 1
    else:
        Capacity[arc] = 2

# Define supply/demand for each node and each commodity
Supply = {}
for i in Nodes:
    for k in Commodities:
        if i == Origin[k]:
            Supply[(i, k)] = -Volume[k]
        elif i == Destination[k]:
            Supply[(i, k)] = Volume[k]
        else:
            Supply[(i, k)] = 0

# Define a set of arc/commodity pairs
ArcCommodities = {(i, j, k) for (i, j) in Arcs for k in Commodities}

# Create the model
from docplex.mp.model import Model
mdl = Model()

# Define decision variables
shipped = mdl.continuous_var_dict(ArcCommodities, lb=0, name='shipped')

# Objective function: Minimize total distance
mdl.minimize(mdl.sum(Distance[i, j] * shipped[i, j, k] for (i, j, k) in shipped))

# Flow balance constraints for each commodity and for each node
for k in Commodities:
    for j in Nodes:
        inflow = mdl.sum(shipped[i, j, k] for i in Nodes if (i, j) in Arcs)
        outflow = mdl.sum(shipped[j, i, k] for i in Nodes if (j, i) in Arcs)
        mdl.add_constraint(inflow - outflow == Supply[j, k])

# Arc capacity constraints (over all commodities)
for (i, j) in Arcs:
    mdl.add_constraint(mdl.sum(shipped[i, j, k] for k in Commodities) <= Capacity[i, j])

# Solve the model
solution = mdl.solve()
mdl.get_solve_details()

# Print the solution
mdl.print_solution()

# Extracting the solution and determining the paths
path_1 = ['1']
path_2 = ['2']

current_node_1 = '1'
current_node_2 = '2'

# Trace the path for shipment C1
while current_node_1 != 'P':
    for (i, j, k) in shipped:
        if k == 'C1' and i == current_node_1 and shipped[i, j, k].solution_value > 0:
            path_1.append(j)
            current_node_1 = j
            break

# Trace the path for shipment C2
while current_node_2 != 'P':
    for (i, j, k) in shipped:
        if k == 'C2' and i == current_node_2 and shipped[i, j, k].solution_value > 0:
            path_2.append(j)
            current_node_2 = j
            break

print("Route 1:", ' -> '.join(path_1))
print("Route 2:", ' -> '.join(path_2))

objective: 634.000
status: OPTIMAL_SOLUTION(2)
  shipped_2_4_C2=1.000
  shipped_10_12_C2=1.000
  shipped_9_12_C1=1.000
  shipped_6_9_C1=1.000
  shipped_4_7_C2=1.000
  shipped_3_6_C1=1.000
  shipped_12_P_C2=1.000
  shipped_7_10_C2=1.000
  shipped_1_3_C1=1.000
  shipped_12_P_C1=1.000
Route 1: 1 -> 3 -> 6 -> 9 -> 12 -> P
Route 2: 2 -> 4 -> 7 -> 10 -> 12 -> P
